In [ ]:
import pandas as pd
from os import walk

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

In [ ]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
#dataFrame.head()

In [ ]:
def dejar_solo_texto(cadena_): # funcion para estandarizar el texto en cada posicion del vector
    
    import nltk
    from nltk.tokenize import RegexpTokenizer    
 
    cadena = "".join(cadena_)
    b = '}{@#$"'                                         #Elimino carecteres especiales indicados enre ''
    for char in b:
        cadena = cadena.replace(char,"")

    r_tokenizer=RegexpTokenizer("(?=\w)[^,]+")           
    
    x=r_tokenizer.tokenize(cadena)
   
    return x


In [ ]:
def final_text(col):
    
    dataFrame[col]=dataFrame[col].apply(dejar_solo_texto)
    dataFrame['amenities_clean']=dataFrame[col].apply(" ".join)
    dataFrame.drop(columns=[col])
    
    return dataFrame['amenities_clean']

In [ ]:
final_text('amenities')

In [ ]:
#cleaning columns
for x in dataFrame.columns:
    print(x)
    values = dataFrame[x][pd.notna(dataFrame[x])].values
    if(len(values) > 0):
        print(values[0])
    else:
        print('NOT_VALUES')
    print('')

In [ ]:
# neighbourhood
# neighbourhood_cleansed
# neighbourhood_group_cleansed
# guests_included
# license
# is_business_travel_ready

cols_to_drop = [
    'market', 'street',
    'listing_url', 'scrape_id', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url',
    'host_name', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode', 'smart_location', 'country_code',
    'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people',
    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
    'requires_license','jurisdiction_names','host_location'
]
# [colName for colName in dataFrame.columns if colName not in cols_to_drop]


# dataFrame.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
import numpy as np

In [ ]:
# drop outliers
from scipy import stats
dataFrame.loc[:,  'price'] = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
dataFrame = dataFrame[dataFrame.price!=0]
dataFrame = dataFrame[(dataFrame.number_of_reviews>2)]
dataFrame = dataFrame[(np.abs(stats.zscore(dataFrame[['price']])) < 3)]

In [ ]:
len(dataFrame)

In [ ]:
#text pipeline steps
from sklearn.base import BaseEstimator, TransformerMixin

#Esta clase simplemente filtra las columnas que se le indica en el constructor
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ]
    
class TextTransformer(BaseEstimator, TransformerMixin):
    
    def __clean_text(self, x):
        for punct in "/-'":
            x = x.replace(punct, ' ')
        for punct in '&':
            x = x.replace(punct, f' {punct} ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~•' + '“”’':
            x = x.replace(punct, '')
        return x.lower()
    
    def __parseTextCols(self, x):
        finalTexts = []
        for i in x:
            if(pd.notna(i) and i not in finalTexts):
                finalTexts.append(i)
        text = self.__clean_text(' '.join(finalTexts))
        return text
    
    def fit(self, X, y = None):
        return self
    
    def transform (self, X, y = None):
        return X.apply(self.__parseTextCols, axis=1)
    
    
from sklearn.feature_extraction.text import TfidfVectorizer
class custom_Tfidf(TfidfVectorizer, TransformerMixin):
    options= {
        'fitSample': 1
    }
    def __init__(self, params, options = None):
        self.vectorizer = TfidfVectorizer(**params)
        if(options != None):
            for key in options.keys():
                self.options[key] = options[key]
        
    def fit(self, X, y = None):
        self.vectorizer.fit(X.sample(frac=self.options['fitSample']))
        return self
    
    def transform(self, X, y = None):
        return self.vectorizer.transform(X)

In [ ]:
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

text_cols = ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about",'amenities_clean']

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')
textVectSettings = {
    'stop_words': stopwords_en,
    'max_df': 0.95,
    'min_df': .05,
    'ngram_range': (1,2),
    'max_features': 300
}

#Pasos para el pipeline Textos
text_pipeline = Pipeline(steps = [
    ( 'text_selector', FeatureSelector(text_cols) ),
    ( 'text_transformer', TextTransformer() ),
    ( 'text_vectorize',  custom_Tfidf(textVectSettings, {'fitSample':1}))
] )

In [ ]:
dummy_cols = ["instant_bookable","is_business_travel_ready","cancellation_policy",
"require_guest_phone_verification",
"require_guest_profile_picture","host_response_time",
"host_is_superhost","host_has_profile_pic","host_identity_verified",
"city","state","property_type","room_type","bed_type"];

from sklearn.impute import SimpleImputer

dummy_pipeline = Pipeline(steps = [
    ('dummy_selector', FeatureSelector(dummy_cols)),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
import numpy as np

class NumericalImputer(BaseEstimator, TransformerMixin):
    def __init__( self, default_strategy = "median"):
        self._default_strategy = default_strategy
        self._default_values = {}
        
    def fit( self, X, y = None ):
        X.host_response_rate = X.host_response_rate.str.replace('%', '').astype(float)
        X.host_acceptance_rate = X.host_acceptance_rate.str.replace('%', '').astype(float)
        
        #Si hay valores infinitos los convertimos en NaN
        X = X.replace( [ np.inf, -np.inf ], np.nan )
        
        for col in X.columns:
            if col=='number_of_reviews_ltm':
                default_value=0;
            elif col=='number_of_reviews':
                default_value=0;
            elif col=='host_listings_count':
                default_value=1;
            elif self._default_strategy=='median':
                default_value=np.median(X[col].dropna())
            elif self._default_strategy=='mode':
                default_value=np.mode(X[col].dropna())
            elif self._default_strategy=='mean':
                default_value=np.mean(X[col].dropna())
            else:
                default_value=np.median(X[col].dropna())
            self._default_values[col]=default_value

        return self 
    
    def transform(self, X, y = None):
        X.host_response_rate = X.host_response_rate.astype(str).str.replace('%', '').astype(float)
        X.host_acceptance_rate = X.host_acceptance_rate.astype(str).str.replace('%', '').astype(float)
        
        for col in X.columns:
            X[col] = X[col].astype(float)
            #Si hay valores infinitos los convertimos en NaN
            X[col] = X[col].replace( [ np.inf, -np.inf ], np.nan)
            X[col].fillna(self._default_values[col],inplace=True)
        return X

class NumericalTransformer(BaseEstimator, TransformerMixin):
    def __init__( self, log_transform = True):
        self._log_transform = log_transform
        
    def fit( self, X, y = None ):
        return self 
    
    def transform(self, X, y = None):
        
        if self._log_transform:
            for col in X.columns:
                colname = col+"_log"
                X.loc[:,colname] = np.log(X[col]+1)
                
        #Retornamos un array de Numpy ?
        return X
    
class NumericalAddFeatures(BaseEstimator, TransformerMixin):
    def __init__( self, bath_per_bed = True, bath_per_bedroom = True ):
        self._bath_per_bed = bath_per_bed
        self._bath_per_bedroom = bath_per_bedroom
        
    def fit( self, X, y = None ):
        return self 
    
    def transform(self, X, y = None):
        if self._bath_per_bedroom:
            X.loc[X['bedrooms']==0,'bedrooms']=1;
            X['bath_per_bedroom'] = X['bathrooms'] / X['bedrooms']
        if self._bath_per_bed:
            X.loc[X['beds']==0,'beds']=1;
            X['bath_per_bed'] = X['bathrooms'] / X['beds']
        
        # ejemplo para clasificar valores
        #pd.cut(df.Age,bins=[0,2,17,65,99],labels=['Toddler/Baby','Child','Adult','Elderly'])
        
        #Retornamos un array de Numpy ?
        return X

In [ ]:
continuos_cols = [
    "reviews_per_month","host_response_rate","host_acceptance_rate",
    "review_scores_communication","review_scores_location","review_scores_value",
    "number_of_reviews_ltm","review_scores_rating","review_scores_cleanliness",
    "review_scores_checkin","availability_30","availability_60","availability_90","availability_365",
    "review_scores_accuracy","minimum_nights","maximum_nights",
    "calculated_host_listings_count","calculated_host_listings_count_entire_homes",
    "calculated_host_listings_count_private_rooms","calculated_host_listings_count_shared_rooms",
    "host_listings_count","number_of_reviews",
    "accommodates","bathrooms","bedrooms","beds","guests_included",
]


numerical_pipeline = Pipeline( steps = [
    ( 'num_selector', FeatureSelector(continuos_cols) ),
    ( 'num_imputer', NumericalImputer(default_strategy = 'median') ),
    ( 'num_transformer', NumericalTransformer() ),
    ( 'std_scaler', StandardScaler() ) 
])

In [ ]:


full_pipeline = FeatureUnion( transformer_list = [ 
    ('numerical_pipeline', numerical_pipeline ),
    ('dummy_pipeline', dummy_pipeline ),
    ('text_pipeline', text_pipeline ),
] )

In [ ]:
from sklearn.model_selection import train_test_split

# y = dataFrame.price.apply(lambda x: x[1:-3].replace(',', '')).astype(int)
y = dataFrame.price
X = dataFrame.drop('price', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train.head()

# full_pipeline.fit(dataFrame)

In [ ]:
full_pipeline.fit(X_train)

In [ ]:
train_data = full_pipeline.transform(X_train)
train_targets = y_train

In [ ]:
train_data.shape

In [ ]:
validation_data = np.log1p(full_pipeline.transform(X_test))
validation_targets = y_test

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import r2_score

In [ ]:
def lgb_r2_score(preds, dtrain):
    labels = dtrain.get_label()
    return 'r2', r2_score(labels, preds), True

In [ ]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'regression',
    'metric': ['lgb_r2_score','l2','AUC'],
    'learning_rate': 0.0005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 1000,
    "n_estimators": 300
}
gbm = lgb.LGBMRegressor(**hyper_params)

In [ ]:
gbm.fit(train_data,train_targets ,
        eval_set=[(validation_data, validation_targets)],
        eval_metric='l1',
        early_stopping_rounds=100)

In [ ]:
y_pred_train = gbm.predict(train_data, num_iteration=gbm.best_iteration_)

In [ ]:
print('The rmse of prediction for train is:', mean_squared_log_error(y_pred_train ,train_targets ))

In [ ]:
y_pred_test = gbm.predict(validation_data, num_iteration=gbm.best_iteration_)

In [ ]:
print('The rmse of prediction for test is:', mean_squared_log_error(y_pred_test ,validation_targets ))

In [ ]:
import numpy

def r2_fit(x, y):
       results = {}

       correlation = numpy.corrcoef(x, y)[0,1]

        # r-squared
       results['R_2'] = correlation**2

       return results

In [ ]:
print('The r-squared of prediction for train is:' ,r2_fit(y_pred_train ,train_targets ))

In [ ]:
print('The r-squared of prediction for test is:', r2_fit(y_pred_test ,validation_targets ))

#### Fuente
https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc 

https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/loss-functions/mean-squared-logarithmic-error-(msle)

https://iartificial.net/error-cuadratico-medio-para-regresion/

http://mkhalusova.github.io/blog/2019/04/17/ml-model-evaluation-metrics-p3#rmsle   

https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html

 